In [4]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session 
from sqlalchemy import create_engine, func
import requests
import json 
import time

In [7]:
from config import gkey

In [8]:
data= "http://www.espn.com/nhl/attendance/_/year/2019"
nhl_attendance = pd.read_html(data)

In [9]:
nhl_attendance=nhl_attendance[0].drop([0])
nhl_attendance.columns=nhl_attendance.iloc[0]
nhl_attendance=nhl_attendance.drop([1])
nhl_attendance=nhl_attendance.drop(['GMS','AVG','PCT','RK'],axis=1)
nhl_attendance.columns = ["team", "total_attendance"]
nhl_attendance=nhl_attendance.sort_values("team")
nhl_attendance=nhl_attendance[nhl_attendance["total_attendance"]!="0"]
nhl_attendance

,team,total_attendance
24,Anaheim,689385
30,Arizona,573552
18,Boston,720165
17,Buffalo,734238
11,Calgary,758550
29,Carolina,587222
2,Chicago,932098
23,Colorado,702446
25,Columbus,682984
14,Dallas,745314


In [10]:
url = "https://en.wikipedia.org/wiki/List_of_National_Hockey_League_arenas"
nhl_stadiums = pd.read_html(url)

In [11]:
nhl_stadiums = nhl_stadiums[0].drop(['Image','Location','Opened','Ref(s)'],axis=1)
nhl_stadiums["Lat"]=""
nhl_stadiums["Lng"]=""
nhl_stadiums

,Arena,Team(s),Capacity,Season of first NHL game,Lat,Lng
0,Amalie Arena,Tampa Bay Lightning,19092,1996–97,,
1,American Airlines Center,Dallas Stars,18532,2001–02,,
2,BB&T Center,Florida Panthers,19250,1998–99,,
3,Bell Centre,Montreal Canadiens,21302,1995–96,,
4,Bell MTS Place,Winnipeg Jets,15321,2011–12,,
5,Bridgestone Arena,Nashville Predators,17113,1998–99,,
6,Canadian Tire Centre,Ottawa Senators,17373,1995–96,,
7,Capital One Arena,Washington Capitals,18506,1997–98,,
8,Climate Pledge Arena,Seattle Kraken,17400,2021 *planned,,
9,Enterprise Center,St. Louis Blues,18096,1994–95,,


In [13]:
params={"key":gkey}
for index,row in nhl_stadiums.iterrows():
    base_url="https://maps.googleapis.com/maps/api/geocode/json"
    arena=row["Arena"]
    params["address"]=arena
    result=requests.get(base_url, params=params)
    result=result.json()
    try:    
        nhl_stadiums.loc[index, "Lat"] = result["results"][0]["geometry"]["location"]["lat"]
        nhl_stadiums.loc[index, "Lng"] = result["results"][0]["geometry"]["location"]["lng"]
        time.sleep(1)
    except (IndexError, KeyError):
        print("results not found...skipping")
        time.sleep(1)
    

In [14]:
nhl_stadiums

,Arena,Team(s),Capacity,Season of first NHL game,Lat,Lng
0,Amalie Arena,Tampa Bay Lightning,19092,1996–97,27.9427,-82.4518
1,American Airlines Center,Dallas Stars,18532,2001–02,32.7903,-96.8101
2,BB&T Center,Florida Panthers,19250,1998–99,26.1585,-80.3255
3,Bell Centre,Montreal Canadiens,21302,1995–96,45.4961,-73.5693
4,Bell MTS Place,Winnipeg Jets,15321,2011–12,49.8927,-97.1438
5,Bridgestone Arena,Nashville Predators,17113,1998–99,36.1592,-86.7785
6,Canadian Tire Centre,Ottawa Senators,17373,1995–96,45.2969,-75.9268
7,Capital One Arena,Washington Capitals,18506,1997–98,38.8981,-77.0208
8,Climate Pledge Arena,Seattle Kraken,17400,2021 *planned,47.6221,-122.354
9,Enterprise Center,St. Louis Blues,18096,1994–95,39.0286,-94.6345
